# Zig-RiR Segmentation Training Notebook

This notebook trains a Zig-RiR model for image segmentation using custom dataset structure:
- train/images (.jpg)
- train/masks (.png binary 0-255)
- val/images (.jpg) 
- val/masks (.png binary 0-255)
- test/images (.jpg) - for prediction only

In [ ]:
# Import required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import argparse
from pathlib import Path
import shutil

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Custom Dataset Class for your data structure
class CustomSegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir=None, transform=None, crop_size=(512, 512), is_test=False):
        self.images_dir = Path(images_dir)
        self.masks_dir = Path(masks_dir) if masks_dir else None
        self.transform = transform
        self.crop_size = crop_size
        self.is_test = is_test
        
        # Get all image files
        self.image_files = sorted([f for f in self.images_dir.glob('*.jpg')])
        
        if not self.is_test:
            # For train/val, check that masks exist
            self.mask_files = []
            for img_file in self.image_files:
                mask_file = self.masks_dir / f"{img_file.stem}.png"
                if mask_file.exists():
                    self.mask_files.append(mask_file)
                else:
                    print(f"Warning: No mask found for {img_file}")
            
            assert len(self.image_files) == len(self.mask_files), "Mismatch between images and masks"
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_files[idx]
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if not self.is_test:
            # Load mask
            mask_path = self.mask_files[idx]
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
            # Convert to binary (0, 1)
            mask = (mask > 127).astype(np.uint8)
        else:
            mask = None
        
        # Resize
        image = cv2.resize(image, self.crop_size, interpolation=cv2.INTER_LINEAR)
        if mask is not None:
            mask = cv2.resize(mask, self.crop_size, interpolation=cv2.INTER_NEAREST)
        
        # Convert to tensor
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        
        if mask is not None:
            mask = torch.from_numpy(mask).long()
            return {'image': image, 'label': mask, 'filename': img_path.name}
        else:
            return {'image': image, 'filename': img_path.name}

In [ ]:
# Configuration
class Config:
    def __init__(self):
        self.data_root = "./data"  # Change this to your data root
        self.crop_size = [512, 512]
        self.nclass = 2  # Background and foreground
        self.batch_size = 4
        self.num_epochs = 50
        self.learning_rate = 0.0003
        self.weight_decay = 0.0001
        self.save_dir = "./checkpoints"
        self.results_dir = "./results"
        self.channels = [64, 128, 256, 512]
        
        # Create directories
        os.makedirs(self.save_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)

config = Config()
print("Configuration loaded")

In [ ]:
# Import and modify the model from your existing files
try:
    from Zig_RiR2d import ZRiR
    print("Successfully imported Zig-RiR model")
except ImportError as e:
    print(f"Error importing Zig-RiR model: {e}")
    print("Please ensure all dependencies are installed and CUDA files are present")
    raise
except Exception as e:
    print(f"Warning: {e}")
    print("Model imported with CPU fallback")
    from Zig_RiR2d import ZRiR

# Loss functions from your existing code
class CrossEntropyLoss(nn.Module):
    def __init__(self, weights=None, ignore_index=255):
        super(CrossEntropyLoss, self).__init__()
        if weights is not None:
            weights = torch.from_numpy(np.array(weights)).float().to(device)
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=ignore_index, weight=weights)

    def forward(self, prediction, label):
        loss = self.ce_loss(prediction, label)
        return loss

class DiceLoss(nn.Module):
    def __init__(self, n_classes):
        super(DiceLoss, self).__init__()
        self.n_classes = n_classes

    def _one_hot_encoder(self, input_tensor):
        tensor_list = []
        for i in range(self.n_classes):
            temp_prob = input_tensor == i * torch.ones_like(input_tensor)
            temp_prob = torch.unsqueeze(temp_prob, 1)
            tensor_list.append(temp_prob)
        output_tensor = torch.cat(tensor_list, dim=1)
        return output_tensor.float()

    def _dice_loss(self, score, target):
        target = target.float()
        smooth = 1e-5
        intersect = torch.sum(score * target)
        y_sum = torch.sum(target * target)
        z_sum = torch.sum(score * score)
        loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
        loss = 1 - loss
        return loss

    def forward(self, inputs, target, weight=None, softmax=True):
        if softmax:
            inputs = torch.softmax(inputs, dim=1)
        target = self._one_hot_encoder(target)
        if weight is None:
            weight = [1] * self.n_classes
        assert inputs.size() == target.size(), 'predict & target shape do not match'
        class_wise_dice = []
        loss = 0.0
        for i in range(0, self.n_classes):
            dice = self._dice_loss(inputs[:, i], target[:, i])
            class_wise_dice.append(1.0 - dice.item())
            loss += dice * weight[i]
        return loss / self.n_classes

# Combined loss
class CombinedLoss(nn.Module):
    def __init__(self, n_classes):
        super(CombinedLoss, self).__init__()
        self.ce_loss = CrossEntropyLoss()
        self.dice_loss = DiceLoss(n_classes)
    
    def forward(self, prediction, target):
        ce = self.ce_loss(prediction, target)
        dice = self.dice_loss(prediction, target)
        return ce + dice

print("Loss functions defined")

In [ ]:
# Create datasets and dataloaders
def create_dataloaders(config):
    # Training dataset
    train_dataset = CustomSegmentationDataset(
        images_dir=os.path.join(config.data_root, "train/images"),
        masks_dir=os.path.join(config.data_root, "train/masks"),
        crop_size=tuple(config.crop_size),
        is_test=False
    )
    
    # Validation dataset
    val_dataset = CustomSegmentationDataset(
        images_dir=os.path.join(config.data_root, "val/images"),
        masks_dir=os.path.join(config.data_root, "val/masks"),
        crop_size=tuple(config.crop_size),
        is_test=False
    )
    
    # Test dataset
    test_dataset = CustomSegmentationDataset(
        images_dir=os.path.join(config.data_root, "test/images"),
        crop_size=tuple(config.crop_size),
        is_test=True
    )
    
    # Dataloaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.batch_size, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=1, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=1, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    return train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset

train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = create_dataloaders(config)
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Initialize model
model = ZRiR(
    channels=config.channels,
    num_classes=config.nclass,
    img_size=config.crop_size[0],
    in_chans=3
).to(device)

# Loss and optimizer
criterion = CombinedLoss(config.nclass)
optimizer = optim.AdamW(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay,
    betas=(0.9, 0.999),
    eps=1e-08
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.num_epochs)

print("Model initialized")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Evaluation metrics (from your existing code)
class Evaluator:
    def __init__(self):
        self.MAE = []
        self.Recall = []
        self.Precision = []
        self.Accuracy = []
        self.Dice = []
        self.IoU = []

    def evaluate(self, pred, gt):
        pred_binary = (pred >= 0.5).float()
        pred_binary_inverse = (pred_binary == 0).float()
        gt_binary = (gt >= 0.5).float()
        gt_binary_inverse = (gt_binary == 0).float()
        
        MAE = torch.abs(pred_binary - gt_binary).mean()
        TP = pred_binary.mul(gt_binary).sum()
        FP = pred_binary.mul(gt_binary_inverse).sum()
        TN = pred_binary_inverse.mul(gt_binary_inverse).sum()
        FN = pred_binary_inverse.mul(gt_binary).sum()
        
        if TP.item() == 0:
            TP = torch.tensor(1.0).to(pred.device)
            
        Recall = TP / (TP + FN + 1e-8)
        Precision = TP / (TP + FP + 1e-8)
        Dice = 2 * Precision * Recall / (Precision + Recall + 1e-8)
        Accuracy = (TP + TN) / (TP + FP + FN + TN + 1e-8)
        IoU = TP / (TP + FP + FN + 1e-8)

        return (MAE.cpu().numpy(), Recall.cpu().numpy(), 
                Precision.cpu().numpy(), Accuracy.cpu().numpy(), 
                Dice.cpu().numpy(), IoU.cpu().numpy())

    def update(self, pred, gt):
        mae, recall, precision, accuracy, dice, iou = self.evaluate(pred, gt)
        self.MAE.append(mae)
        self.Recall.append(recall)
        self.Precision.append(precision)
        self.Accuracy.append(accuracy)
        self.Dice.append(dice)
        self.IoU.append(iou)

    def get_metrics(self):
        return {
            'MAE': np.mean(self.MAE) * 100,
            'Recall': np.mean(self.Recall) * 100,
            'Precision': np.mean(self.Precision) * 100,
            'Accuracy': np.mean(self.Accuracy) * 100,
            'Dice': np.mean(self.Dice) * 100,
            'IoU': np.mean(self.IoU) * 100
        }

print("Evaluator class defined")

In [ ]:
# Training function
def train_one_epoch(model, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0.0
    num_batches = len(train_loader)
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.num_epochs}')
    
    for batch_idx, batch in enumerate(pbar):
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Update progress bar
        pbar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Avg Loss': f'{total_loss/(batch_idx+1):.4f}',
            'LR': f'{optimizer.param_groups[0]["lr"]:.6f}'
        })
    
    return total_loss / num_batches

print("Training function defined")

In [ ]:
# Validation function
def validate(model, val_loader):
    model.eval()
    evaluator = Evaluator()
    total_loss = 0.0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # Get predictions
            predictions = torch.argmax(outputs, dim=1)
            pred_binary = predictions.float()
            gt_binary = labels.float()
            
            # Update evaluator
            evaluator.update(pred_binary, gt_binary)
    
    metrics = evaluator.get_metrics()
    avg_loss = total_loss / len(val_loader)
    
    return avg_loss, metrics

print("Validation function defined")

In [ ]:
# Training loop
best_dice = 0.0
train_losses = []
val_losses = []
val_metrics_history = []

print("Starting training...")

for epoch in range(config.num_epochs):
    # Training
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, epoch)
    train_losses.append(train_loss)
    
    # Validation
    val_loss, val_metrics = validate(model, val_loader)
    val_losses.append(val_loss)
    val_metrics_history.append(val_metrics)
    
    # Update learning rate
    scheduler.step()
    
    # Print metrics
    print(f"\nEpoch {epoch+1}/{config.num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val Metrics: Dice: {val_metrics['Dice']:.2f}, IoU: {val_metrics['IoU']:.2f}, Acc: {val_metrics['Accuracy']:.2f}")
    
    # Save best model
    if val_metrics['Dice'] > best_dice:
        best_dice = val_metrics['Dice']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_dice': best_dice,
            'val_metrics': val_metrics
        }, os.path.join(config.save_dir, 'best_model.pth'))
        print(f"New best model saved with Dice: {best_dice:.2f}")
    
    # Save checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_metrics': val_metrics
        }, os.path.join(config.save_dir, f'checkpoint_epoch_{epoch+1}.pth'))

print(f"\nTraining completed! Best Dice: {best_dice:.2f}")

In [ ]:
# Plot training curves
plt.figure(figsize=(15, 5))

# Loss curves
plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Dice score
plt.subplot(1, 3, 2)
dice_scores = [m['Dice'] for m in val_metrics_history]
plt.plot(dice_scores, label='Dice Score', color='green')
plt.title('Validation Dice Score')
plt.xlabel('Epoch')
plt.ylabel('Dice (%)')
plt.legend()
plt.grid(True)

# IoU score
plt.subplot(1, 3, 3)
iou_scores = [m['IoU'] for m in val_metrics_history]
plt.plot(iou_scores, label='IoU Score', color='orange')
plt.title('Validation IoU Score')
plt.xlabel('Epoch')
plt.ylabel('IoU (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(config.results_dir, 'training_curves.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Load best model for prediction
def load_best_model():
    model = ZRiR(
        channels=config.channels,
        num_classes=config.nclass,
        img_size=config.crop_size[0],
        in_chans=3
    ).to(device)
    
    checkpoint = torch.load(os.path.join(config.save_dir, 'best_model.pth'))
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Loaded best model with Dice: {checkpoint['best_dice']:.2f}")
    return model

best_model = load_best_model()

In [ ]:
# Prediction function
def predict_test_set(model, test_loader, output_dir):
    model.eval()
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Predicting on {len(test_dataset)} test images...")
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Predicting'):
            images = batch['image'].to(device)
            filenames = batch['filename']
            
            # Get predictions
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            
            # Convert to numpy and save
            for i, filename in enumerate(filenames):
                pred_mask = predictions[i].cpu().numpy().astype(np.uint8) * 255
                
                # Save prediction
                output_path = os.path.join(output_dir, f"{Path(filename).stem}_pred.png")
                cv2.imwrite(output_path, pred_mask)
    
    print(f"Predictions saved to {output_dir}")

# Create predictions directory and run prediction
predictions_dir = os.path.join(config.results_dir, "test_predictions")
predict_test_set(best_model, test_loader, predictions_dir)

In [ ]:
# Visualize some predictions
def visualize_predictions(test_loader, model, num_samples=5):
    model.eval()
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 6))
    
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            if i >= num_samples:
                break
                
            images = batch['image'].to(device)
            filename = batch['filename'][0]
            
            # Get prediction
            outputs = model(images)
            prediction = torch.argmax(outputs, dim=1)
            
            # Convert to numpy for visualization
            image_np = images[0].cpu().permute(1, 2, 0).numpy()
            pred_np = prediction[0].cpu().numpy()
            
            # Plot original image
            axes[0, i].imshow(image_np)
            axes[0, i].set_title(f'Input: {filename}')
            axes[0, i].axis('off')
            
            # Plot prediction
            axes[1, i].imshow(pred_np, cmap='gray')
            axes[1, i].set_title('Prediction')
            axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(config.results_dir, 'sample_predictions.png'), dpi=300, bbox_inches='tight')
    plt.show()

visualize_predictions(test_loader, best_model)

# Training Summary

The notebook has completed the following steps:
1. ✅ Loaded and preprocessed your custom dataset structure
2. ✅ Trained the Zig-RiR model with combined Dice + CrossEntropy loss
3. ✅ Validated the model and tracked metrics
4. ✅ Saved the best model based on Dice score
5. ✅ Generated predictions on test set
6. ✅ Created visualizations of training progress and sample predictions

## Output Files:
- **Checkpoints**: `./checkpoints/best_model.pth` 
- **Predictions**: `./results/test_predictions/` (contains prediction masks)
- **Visualizations**: `./results/training_curves.png`, `./results/sample_predictions.png`

## Next Steps:
- Adjust hyperparameters if needed
- Experiment with data augmentation
- Try different loss functions or optimizers
- Evaluate on additional metrics